In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import deepmatcher as dm
import pandas as pd
import numpy as np
import scipy.spatial as spatial
from tqdm import tqdm
import shutil
import pickle
import os

## Build word dictionary

In [3]:
from utils.attacker import getDictionary

In [4]:
GLOVE_PATH = 'glove.840B.300d.txt'

In [5]:
train_df = pd.read_csv('Structured/itunes-amazon/merged_train.csv')
validation_df = pd.read_csv('Structured/itunes-amazon/merged_valid.csv')
test_df = pd.read_csv('Structured/itunes-amazon/merged_test.csv')

In [86]:
wordDict = getDictionary([train_df,validation_df,test_df])
len(wordDict)

1828

## Find closest words with FastText/ Glove

In [6]:
from utils.glove_utils import loadGloveModel,getTopClosest

In [7]:
gloveModel2 = loadGloveModel('counter-fitted-vectors.txt')
embeddings2 = list(gloveModel2.values())
embedding_mat2 = np.array(embeddings2)

Loading Glove Model
Done. 65713  words loaded!


In [91]:
closest_words_map,notfound = getTopClosest(wordDict,gloveModel2,embedding_matrix=embedding_mat2
                                           ,maxDistance=0.25)

100%|██████████| 1828/1828 [00:49<00:00, 37.13it/s]


In [92]:
len(notfound)

407

In [ ]:
def getClosestFastText(dictionary,ft_vectors,minSimilarity):
    closest = {}
    ##not necessary
    for word in tqdm(dictionary.keys()):
        closestWords = ft_vectors.most_similar(positive=[word],topn=1000)
        closestWords = list(filter(lambda wd:wd[1]>=minSimilarity,closestWords))
        closest[word] = closestWords
    return closest

In [ ]:
closestForFastText = getClosestFastText(wordDict,wv,0.5)

In [ ]:
with open('closest_words_ft.pickle', 'wb') as handle:
    pickle.dump(closestForFastText, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('closest_words.pickle', 'wb') as handle:
    pickle.dump(closest_words_map, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('notfound_words.data','wb') as handle:
    pickle.dump(notfound,handle)

In [ ]:
with open('closest_words.pickle', 'rb') as handle:
    closest_words_map = pickle.load(handle)

In [ ]:
with open('notfound_words.data', 'rb') as handle:
    notfound = pickle.load(handle)

## Attack single attribute

In [17]:
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

### Itunes-Amazon

In [14]:
from utils.attacker import attackSample

In [93]:
attributes = [col for col in list(test_df) if col not in ['id','label']]
itunesamazon_model = dm.MatchingModel(attr_summarizer='hybrid')
itunesamazon_model.load_state('models/itunes_amazon_hybrid.pth')

In [107]:
x1_s = pd.Series(x1)
x2_s = pd.Series(x2)
df = pd.DataFrame(data=[x1_s,x2_s])
df.head()

,ltable_Song_Name,ltable_Artist_Name,ltable_Album_Name,ltable_Genre,ltable_Price,ltable_CopyRight,ltable_Time,ltable_Released,rtable_Song_Name,rtable_Artist_Name,rtable_Album_Name,rtable_Genre,rtable_Price,rtable_CopyRight,rtable_Time,rtable_Released
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [108]:
test_index = test_df.index
attack_successfull = []
for idx in test_index:
    curr_attack = attackSample(test_df,idx,attributes,itunesamazon_model,closest_words_map,
                                     notfound,stop_words)
    curr_attack = pd.Series(curr_attack)
    attack_successfull.append(curr_attack)
attack_df = pd.DataFrame(attack_successfull)


Reading and processing data from "temp/test.csv"
0% [# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 5
Finished Epoch 5 || Run Time:    0.1 | Load Time:    0.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00




Reading and processing data from "temp/test.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 5


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Reading and processing data from "temp/test.csv"


...

===>  PREDICT Epoch 5
Finished Epoch 5 || Run Time:    0.1 | Load Time:    0.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00




Reading and processing data from "temp/test.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 5


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 5 || Run Time:    0.7 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [111]:
for col in list(attack_df):
    print(attack_df[col].value_counts())

False    109
Name: ltable_Song_Name, dtype: int64
False    108
True       1
Name: ltable_Artist_Name, dtype: int64
False    109
Name: ltable_Album_Name, dtype: int64
False    109
Name: ltable_Genre, dtype: int64
False    109
Name: ltable_Price, dtype: int64
False    109
Name: ltable_CopyRight, dtype: int64
False    87
True     22
Name: ltable_Time, dtype: int64
False    109
Name: ltable_Released, dtype: int64
False    108
True       1
Name: rtable_Song_Name, dtype: int64
False    108
True       1
Name: rtable_Artist_Name, dtype: int64
False    109
Name: rtable_Album_Name, dtype: int64
False    109
Name: rtable_Genre, dtype: int64
False    109
Name: rtable_Price, dtype: int64
False    109
Name: rtable_CopyRight, dtype: int64
False    91
True     18
Name: rtable_Time, dtype: int64
False    109
Name: rtable_Released, dtype: int64


### Beer Advo

In [112]:
train_df = pd.read_csv('Structured/Beer/merged_train.csv')
validation_df = pd.read_csv('Structured/Beer/merged_valid.csv')
test_df = pd.read_csv('Structured/Beer/merged_test.csv')

In [113]:
wordDict = getDictionary([train_df,validation_df,test_df])
len(wordDict)

1211

In [114]:
closest_words_map,notfound = getTopClosest(wordDict,gloveModel2,embedding_matrix=embedding_mat2
                                           ,maxDistance=0.25)

100%|██████████| 1211/1211 [00:32<00:00, 37.67it/s]


In [115]:
beer_model = dm.MatchingModel(attr_summarizer='hybrid')
beer_model.load_state('models/beer_hybrid.pth')

In [117]:
attributes = [col for col in list(test_df) if col not in ['id','label']]

In [118]:
test_index = test_df.index
attack_successfull = []
for idx in test_index:
    curr_attack = attackSample(test_df,idx,attributes,beer_model,closest_words_map,
                                     notfound,stop_words)
    curr_attack = pd.Series(curr_attack)
    attack_successfull.append(curr_attack)
attack_df = pd.DataFrame(attack_successfull)


Reading and processing data from "temp/test.csv"
0% [# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 5



Reading and processing data from "temp/test.csv"


Finished Epoch 5 || Run Time:    0.1 | Load Time:    0.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



0% [############################# ] 100% | ETA: 00:00:00

...

===>  PREDICT Epoch 5
Finished Epoch 5 || Run Time:    0.1 | Load Time:    0.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00




Reading and processing data from "temp/test.csv"
0% [############################# ] 100% | ETA: 00:00:00

===>  PREDICT Epoch 5


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 5 || Run Time:    0.5 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s:   0.00



In [119]:
for col in list(attack_df):
    print(attack_df[col].value_counts())

False    83
True      8
Name: ltable_Beer_Name, dtype: int64
False    90
True      1
Name: ltable_Brew_Factory_Name, dtype: int64
False    90
True      1
Name: ltable_Style, dtype: int64
False    89
True      2
Name: ltable_ABV, dtype: int64
False    84
True      7
Name: rtable_Beer_Name, dtype: int64
False    89
True      2
Name: rtable_Brew_Factory_Name, dtype: int64
False    90
True      1
Name: rtable_Style, dtype: int64
False    89
True      2
Name: rtable_ABV, dtype: int64


### Walmart-Amazon

In [8]:
train_df = pd.read_csv('Structured/Walmart-Amazon/merged_train.csv')
validation_df = pd.read_csv('Structured/Walmart-Amazon/merged_valid.csv')
test_df = pd.read_csv('Structured/Walmart-Amazon/merged_test.csv')

In [9]:
wordDict = getDictionary([train_df,validation_df,test_df])
len(wordDict)

11465

In [10]:
closest_words_map,notfound = getTopClosest(wordDict,gloveModel2,embedding_matrix=embedding_mat2
                                           ,maxDistance=0.25)

100%|██████████| 11465/11465 [01:40<00:00, 113.87it/s]


In [11]:
walmartamazon_model = dm.MatchingModel(attr_summarizer='hybrid')
walmartamazon_model.load_state('models/walmartamazon_hybrid.pth')

In [12]:
attributes = [col for col in list(test_df) if col not in ['id','label']]

In [ ]:
%%capture
test_index = test_df.index
attack_successfull = []
for idx in test_index:
    curr_attack = attackSample(test_df,idx,attributes,walmartamazon_model,closest_words_map,
                                     notfound,stop_words)
    curr_attack = pd.Series(curr_attack)
    attack_successfull.append(curr_attack)
attack_df = pd.DataFrame(attack_successfull)